In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import pickle

os.chdir('C:/Users/Kanav/Documents/urlclassifier/scripts')
import data_creation as d

In [2]:

os.chdir('C:/Users/Kanav/Documents/urlclassifier/models')

In [3]:
order = ['bodyLength', 'bscr', 'dse', 'dsr', 'entropy', 'hasHttp', 'hasHttps',
       'has_ip', 'numDigits', 'numImages', 'numLinks', 'numParams',
       'numTitles', 'num_%20', 'num_@', 'sbr', 'scriptLength', 'specialChars',
       'sscr', 'urlIsLive', 'urlLength']

In [6]:
a = d.UrlFeaturizer('http://astore.amazon.co.uk/allezvinsfrenchr/detail/1904010202/026-8324244-9330038').run()
test = []
for i in order:
    test.append(a[i])
encoder = LabelEncoder()
encoder.classes_ = np.load('lblenc_v1.npy',allow_pickle=True)
scalerfile = 'scaler_v1.sav'
scaler = pickle.load(open(scalerfile, 'rb'))

In [7]:
test = pd.DataFrame(test).replace(True,1).replace(False,0).to_numpy(dtype="float32").reshape(1,-1)
test = scaler.transform(test)

C:\Users\Kanav\AppData\Local\Temp\ipykernel_8876\1755746282.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = pd.DataFrame(test).replace(True,1).replace(False,0).to_numpy(dtype="float32").reshape(1,-1)
c:\Users\Kanav\Documents\urlclassifier\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [11]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="tflite_quant_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], test)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted = np.argmax(output_data,axis=1)

In [12]:
print(encoder.inverse_transform(predicted)[0])

Spam
